In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
cookie = "Oxford_AcademicMachineID=636567559735400801; _ga=GA1.3.1653056694.1521159178; SaneID=75OQuDXjiiY-awZm3Sy; __gads=ID=18251e3c3f2021e5:T=1521159178:S=ALNI_MbulwuKwqC1zJr93qQz-ie5imB7gw; oup-cookie=1_15-3-2018; OUP_SessionId=alzeb54qfp4ul0eazvpa0oe4; _gid=GA1.3.243705058.1523734574; JSESSIONID=B0BB53699D3F70D058C3265070B819DE; NSC_OSO_TWD_THL_443=ffffffff09099b1b45525d5f4f58455e445a4a4229a0; __atuvc=54%7C11%2C25%7C12%2C78%7C13%2C0%7C14%2C16%7C15; __atuvs=5ad28e0f4fc0fb89002"
headers = {
    'User-Agent': 'Chrome/64.0.3282.168',
    'Cookie' : cookie}
session = requests.session()


In [3]:
#import urllib.request
#d = urllib.request.urlopen("http://www.google.co.uk")
#d.getheader('Set-Cookie')

In [4]:
def get_url_source_html(url):
    response = session.get(url, headers=headers)
    redirected_url = response.history[-1].url
    print(redirected_url)

    response = session.get(redirected_url, headers=headers)
    return response.text

In [5]:
def get_soup(myarticle):
    return BeautifulSoup(myarticle, "lxml")

def strip_hyperlinks(mysoup):
    for a in mysoup.findAll('a'):
        a.replaceWith(" %s " % a.string)
    return(mysoup)

def strip_references(mysoup):
    for div in mysoup.find_all("div", {'class':'ref-list'}): 
        div.decompose()
    return(mysoup)

def strip_copywrite(mysoup):
    for div in mysoup.find_all("div", {'class':'copyright copyright-statement'}): 
        div.decompose()
    return(mysoup)

def strip_tables(mysoup):
    codetags = mysoup.find_all('table')
    for codetag in codetags:
       codetag.extract()
    return(mysoup)

def strip_rnt(mysoup):
    return(mysoup.get_text().replace('\r', ' ').replace('\n', ' ').replace('\t', ' '))
 
def clean_article(mysoup):
    #soup = get_soup(myarticle)
    mysoup = strip_hyperlinks(mysoup)
    mysoup = strip_references(mysoup)
    mysoup = strip_copywrite(mysoup)
    mysoup = strip_tables(mysoup)
    soup_text = strip_rnt(mysoup)
    return soup_text


In [6]:
def parse_source_html(source_html):
    soup = BeautifulSoup(source_html, "html.parser")

    raw_html.append(soup.get_text())
    
    citation_date = soup.find("div", {"class": "citation-date"}).text
    citation_dates.append(citation_date)

    citation = soup.find(attrs={'class': 'ww-citation-primary'}).text
    citations.append(citation)

    article_body = soup.find("div", {"class": "article-body"})
    articles.append(article_body)
    
    types.append(soup.find(attrs={'property': 'og:type'}).attrs['content'])
    titles.append(soup.find(attrs={'property': 'og:title'}).attrs['content'])
    descriptions.append(soup.find(attrs={'property': 'og:description'}).attrs['content'])
    urls.append(soup.find(attrs={'property': 'og:url'}).attrs['content'])
    updated_times.append(soup.find(attrs={'property': 'og:updated_time'}).attrs['content'])
    site_names.append(soup.find(attrs={'property': 'og:site_name'}).attrs['content'])
    

    article = clean_article(article_body)
    clean_articles.append(article)
    

In [9]:
test=False
citations = []
citation_dates = []
articles = []
clean_articles = []
types = []
titles = []
descriptions = []
urls = []
updated_times = []
site_names = []
raw_html = []

issueListFilename = "2001-2008URLs.csv"
get_year = 2002
count = 0

def process_url(my_url):
    global count
    print(str(count) + ": " + my_url)
    source_html = get_url_source_html(my_url)
    time.sleep(5)
    #print(source_html)
    parse_source_html(source_html)
    time.sleep(10)
    count += 1
    
     
if test != True:
    df = pd.read_csv(issueListFilename)
    df = df[df.year == get_year] 
    print(len(df.index))
    df.url.apply(lambda x: process_url(x))
    
else:
    url = 'https://doi.org/10.1093/geront/gnx189'
    print(url)
    source_html = get_url_source_html(url)
    parse_source_html(source_html)


152
0: https://doi.org/10.1093/geront/42.1.4
https://academic.oup.com/gerontologist/article/42/1/4/641484/Hydra-RevisitedSubstituting-Formal-for-Self-and
1: https://doi.org/10.1093/geront/42.1.17
https://academic.oup.com/gerontologist/article/42/1/17/641500/A-Prospective-Study-of-the-Relationship-Between
2: https://doi.org/10.1093/geront/42.1.24
https://academic.oup.com/gerontologist/article/42/1/24/641493/Maintaining-Functional-Independence-in-Elderly
3: https://doi.org/10.1093/geront/42.1.32
https://academic.oup.com/gerontologist/article/42/1/32/641496/Consumer-Direction-in-Managed-LongTerm-CareAn
4: https://doi.org/10.1093/geront/42.1.39
https://academic.oup.com/gerontologist/article/42/1/39/641486/A-SocialCognitive-Model-to-Predict-the-Use-of
5: https://doi.org/10.1093/geront/42.1.51
https://academic.oup.com/gerontologist/article/42/1/51/641490/A-Comparison-of-State-Advance-Directive-Documents
6: https://doi.org/10.1093/geront/42.1.61
https://academic.oup.com/gerontologist/article/

59: https://doi.org/10.1093/geront/42.4.507
https://academic.oup.com/gerontologist/article/42/4/507/686611/Pain-Reports-by-Older-Hospice-Cancer-Patients-and
60: https://doi.org/10.1093/geront/42.4.515
https://academic.oup.com/gerontologist/article/42/4/515/686610/Effects-of-Age-and-Causal-Attribution-to-Aging-on
61: https://doi.org/10.1093/geront/42.4.522
https://academic.oup.com/gerontologist/article/42/4/522/686616/The-Effect-of-Widowhood-on-Older-Adults-Social
62: https://doi.org/10.1093/geront/42.4.534
https://academic.oup.com/gerontologist/article/42/4/534/686620/Development-Reliability-and-Validity-of-the
63: https://doi.org/10.1093/geront/42.4.543
https://academic.oup.com/gerontologist/article/42/4/543/686622/Workers-Ignorance-of-Retirement-Benefits
64: https://doi.org/10.1093/geront/42.4.552
https://academic.oup.com/gerontologist/article/42/4/552/686618/The-Influence-of-CommercialGrade-Carpet-on
65: https://doi.org/10.1093/geront/42.4.560
http://academic.oup.com/gerontologist/a

115: https://doi.org/10.1093/geront/42.suppl_3.49
https://academic.oup.com/gerontologist/article/42/suppl_3/49/569214/A-CommentarySocial-and-Cultural-Determinants-of
116: https://doi.org/10.1093/geront/42.suppl_3.54
https://academic.oup.com/gerontologist/article/42/suppl_3/54/569218/What-Impact-Do-Setting-and-Transitions-Have-on-the
117: https://doi.org/10.1093/geront/42.suppl_3.68
https://academic.oup.com/gerontologist/article/42/suppl_3/68/569216/A-CommentaryWhere-to-Live-While-Dying
118: https://doi.org/10.1093/geront/42.suppl_3.71
https://academic.oup.com/gerontologist/article/42/suppl_3/71/569221/Measurement-of-Quality-of-Care-and-Quality-of-Life
119: https://doi.org/10.1093/geront/42.suppl_3.81
https://academic.oup.com/gerontologist/article/42/suppl_3/81/569220/A-CommentaryCrossCultural-QualityofLife-Assessment
120: https://doi.org/10.1093/geront/42.suppl_3.86
https://academic.oup.com/gerontologist/article/42/suppl_3/86/569224/Research-Design-in-EndofLife-ResearchState-of
121: ht

https://doi.org/10.1093/geront/41.6.843
https://academic.oup.com/gerontologist/article/41/6/843/555778/Audiovisual-Reviews


In [10]:
df = pd.DataFrame({'citation_date': citation_dates,
                    'citation': citations,
                    'title': titles,
                    'type': types,
                    'description': descriptions,
                    'url': urls,
                    'updated_time': updated_times,
                    'site_name': site_names,
                    'article': articles,
                    'clean_article': clean_articles,
                    'raw_html' : raw_html
                    },columns = ['citation_date', 'citation', 'title', 'type', 'description', 'url', 'updated_time', 'site_name', 'article', 'clean_article', 'raw_html'])
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 11 columns):
citation_date    152 non-null object
citation         152 non-null object
title            152 non-null object
type             152 non-null object
description      152 non-null object
url              152 non-null object
updated_time     152 non-null object
site_name        152 non-null object
article          152 non-null object
clean_article    152 non-null object
raw_html         152 non-null object
dtypes: object(11)
memory usage: 13.1+ KB
None


,citation_date,citation,title,type,description,url,updated_time,site_name,article,clean_article,raw_html
0,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Hydra Revisited: Substituting Formal for Self-...,article,Abstract. Purpose: In response to concerns am...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
1,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Prospective Study of the Relationship Between ...,article,Abstract. Purpose: To identify the most commo...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
2,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Maintaining Functional Independence in Elderly...,article,Abstract. Purpose: We investigated whether he...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
3,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Consumer Direction in Managed Long-Term Care: ...,article,Abstract. Purpose: This article presents resu...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
4,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Social–Cognitive Model to Predict the Use of A...,article,Abstract. Purpose: To provide insight into th...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
5,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Comparison of State Advance Directive Document...,article,Abstract. Purpose: Advance directive (AD) doc...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
6,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Strategies Used by Families to Simplify Tasks ...,article,Abstract. Purpose: Little is known about the ...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
7,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Private Prayer and Optimism in Middle-Aged and...,article,Abstract. Purpose: This study investigated th...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
8,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...",Effect of Religious Coping on Caregiving Appra...,article,Abstract. Purpose: This article explores the ...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n ...
9,01 February 2002,"The Gerontologist, Volume 42, Issue 1, 1 Febru...","Comparison of Young, Middle-Aged, and Older Ad...",article,Abstract. Purpose: Pathological gambling is a...,https://academic.oup.com/gerontologist/article...,2002-02-01,OUP Academic,"<div class=""article-body""> <div class=""toolbar...",None None Article contents Fig...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r

In [11]:
df["clean_article"][0]

'      None     None     Article contents   Figures & tables   Video   Audio   Supplementary Data      None      None   Citation Margaret J. Penning;  Hydra Revisited: Substituting Formal for Self- and Informal In-Home Care Among Older Adults With Disabilities, The Gerontologist, Volume 42, Issue 1, 1 February 2002, Pages 4–16,  https://doi.org/10.1093/geront/42.1.4  Download citation file:   RIS (Zotero)   EndNote   BibTex   Medlars   ProCite   RefWorks   Reference Manager    © 2018 Oxford University Press  None          Permissions       None     Email   Twitter   Facebook            Navbar Search Filter   All     All  Journals   Mobile Microsite Search Term           None          Register             Close   search filter   All     All  Journals   search input      Advanced Search               Abstract Purpose: In response to concerns among policymakers and others that increases in the availability of publicly funded formal services will lead to reductions in self- and informal ca

In [12]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df["clean_article"].apply(lambda x: my_print(x))

0 56106
1 31506
2 39509
3 34106
4 55876
5 30413
6 42154
7 52525
8 43894
9 31871
10 39848
11 19290
12 33946
13 23637
14 798
15 808
16 796
17 13496
18 5776
19 10964
20 8004
21 50093
22 42138
23 44688
24 38454
25 30316
26 41578
27 28401
28 53534
29 106568
30 20196
31 795
32 805
33 3940
34 14499
35 37483
36 40280
37 27525
38 69265
39 33357
40 36667
41 30777
42 72016
43 37947
44 23226
45 23781
46 29024
47 27628
48 33254
49 794
50 804
51 861
52 34601
53 39559
54 35055
55 59777
56 49623
57 42272
58 39780
59 34866
60 31075
61 55306
62 39389
63 43859
64 35512
65 1195
66 1168
67 1148
68 1177
69 861
70 1159
71 1185
72 834
73 45515
74 42677
75 43532
76 29107
77 59713
78 44280
79 46093
80 33110
81 25505
82 40208
83 52025
84 34471
85 28527
86 797
87 807
88 2295376
89 90436
90 313085
91 90383
92 2409
93 29356
94 1822
95 7914
96 1699
97 5748
98 1084
99 1475
100 2390
101 1093
102 3868
103 5868
104 2606
105 2993
106 43649
107 1055
108 779
109 27721
110 44523
111 21030
112 46759
113 31716
114 41569
115 2

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
122    None
123    None
124    None
125    None
126    None
127    None
128    None
129    None
130    None
131    None
132    None
133    None
134    None
135    None
136    None
137    None
138    None
139    None
140    None
141    None
142    None
143    None
144    None
145    None
146    None
147    None
148    None
149    None
150    None
151    None
Name: clean_article, Length: 152, dtype: object

In [13]:
df.loc[114]

citation_date                                      01 October 2002
citation         The Gerontologist, Volume 42, Issue suppl_3, 1...
title            What Is Known About the Economics of End-of-Li...
type                                                       article
description      Abstract.  Purpose: This article reviews the s...
url              https://academic.oup.com/gerontologist/article...
updated_time                                            2002-10-01
site_name                                             OUP Academic
article          <div class="article-body">
<div class="toolbar...
clean_article          None     None     Article contents   Fig...
raw_html         \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n  ...
Name: 114, dtype: object

In [14]:
count = 0
remove_indices = []
def get_short_articles(my_x):
    global remove_indices, count
    if len(my_x) < 500:
        remove_indices.append(count)
    count += 1

df["clean_article"].apply(lambda x: get_short_articles(x))
print("Initial set: ", len(df))
print("Remove set: " + str(len(remove_indices)))
print(remove_indices)

df_reduced = df.drop(remove_indices)
print("After removal: " + str(len(df_reduced)))

Initial set:  152
Remove set: 0
[]
After removal: 152


In [69]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df_reduced["clean_article"].apply(lambda x: my_print(x))

0 50680
1 33637
2 46824
3 40950
4 37925
5 44231
6 33388
7 50781
8 26235
9 28621
10 33756
11 37017
12 60989
13 25018
14 14827
15 34607
16 54476
17 30502
18 36639
19 12371
20 45099
21 41377
22 45234
23 28145
24 45491
25 39582
26 31135
27 24304
28 24878
29 5892
30 3529
31 12309
32 60923
33 9541
34 8980
35 47923
36 47798
37 49098
38 42866
39 39027
40 34893
41 43015
42 42815
43 30926
44 23071
45 16578
46 32842
47 57927
48 54143
49 34889
50 44779
51 39338
52 42300
53 22860
54 39953
55 54964
56 31879
57 30687
58 11357
59 9183
60 8464
61 1968
62 24756
63 36549
64 26073
65 26222
66 53442
67 33264
68 45099
69 41856
70 20853
71 60985
72 33468
73 32416
74 7675
75 9065
76 25316
77 44006
78 33797
79 34468
80 22634
81 44253
82 42550
83 43069
84 47832
85 23104
86 54197
87 33629
88 2857


0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
41     None
       ... 
83     None
84     None
85     None
86     None
87     None
88     None
89     None
90     None
91     None
92     None
93     None
94     None
95     None
96     None
97     None
98     None
99     None
102    None
103    None
104    None
105    None
106    None
107    None
108    None
109    None
110    None
111    None
112    None
113    None
114    None
Name: clean_article, Length: 89, dtype: object

In [15]:
import os
base_filename = 'articles2002.txt'
with open(os.path.join(base_filename),'w') as outfile:
    df_reduced["clean_article"].apply(lambda x: outfile.write(x + '\n'))
outfile.close()
num_lines = sum(1 for line in open(base_filename))
print(num_lines)
            

152


In [3]:
import sys
sys.setrecursionlimit(10000)
df_reduced.to_pickle("data_2002.pkl.gzip",compression='gzip')

NameError: name 'df_reduced' is not defined

In [1]:
df_reduced.to_csv("articles2002.csv")

NameError: name 'df_reduced' is not defined